In [2]:
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt

import gensim

from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve, precision_score, recall_score
from sklearn.metrics import classification_report

from sklearn.metrics import classification_report, f1_score


#text is already cleaned.
#assign cleaned data to these variables.
train_data_path = 'cleaned_data/cleaned_train_data_for_subtask1.csv'
test_data_path = 'cleaned_data/cleaned_test_data_for_subtask1.csv'
#read files.
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

print("Train set:"% train_data.columns, train_data.shape, len(train_data)) 
print("Test set:"% test_data.columns, test_data.shape, len(test_data)) 



Train set: (20974, 9) 20974
Test set: (4997, 9) 4997


In [3]:
#prepare train and test data.
X_train = train_data['#2_tweet_clean_farasaV0']
y_train = train_data['#classes_id']
X_test = test_data['#2_tweet_clean_farasaV0']
y_test = test_data['#classes_id']

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

max_no = 50000
sql_len = 300
dim = 64

tokenizer = Tokenizer(num_words=max_no, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=False)
tokenizer.fit_on_texts(X_train.values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

tokenizer = Tokenizer(num_words=max_no, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=False)
tokenizer.fit_on_texts(X_test.values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 32841 unique tokens.
Found 12650 unique tokens.


In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


X_tr = tokenizer.texts_to_sequences(X_train.values)
X_tr = pad_sequences(X_tr, maxlen=sql_len)
print('Shape of data tensor:', X_tr.shape)

X_te = tokenizer.texts_to_sequences(X_test.values)
X_te = pad_sequences(X_te, maxlen=sql_len)
print('Shape of data tensor:', X_te.shape)


Shape of data tensor: (20974, 300)
Shape of data tensor: (4997, 300)


In [7]:
Y_tr = pd.get_dummies(y_train).values
print('Shape of label tensor:', Y_tr.shape)

Y_te = pd.get_dummies(y_test).values
print('Shape of label tensor:', Y_te.shape)

Shape of label tensor: (20974, 21)
Shape of label tensor: (4997, 21)


In [10]:
import tensorflow as tf

from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense
#import tensorsflow_addons as tfa
model = tf.keras.Sequential()
model.add(Embedding(max_no, dim, input_shape=(sql_len,)))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(21, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 20
batch_size = 8

history = model.fit(X_tr, Y_tr, epochs=epochs, batch_size=batch_size,validation_split=0.1)

Epoch 1/20


InternalError: 2 root error(s) found.
  (0) Internal:  Blas GEMM launch failed : a.shape=(8, 100), b.shape=(100, 100), m=8, n=100, k=100
	 [[{{node sequential_1/lstm_1/while/body/_1/sequential_1/lstm_1/while/lstm_cell_1/MatMul_6}}]]
	 [[gradient_tape/sequential_1/embedding_1/embedding_lookup/Reshape/_116]]
  (1) Internal:  Blas GEMM launch failed : a.shape=(8, 100), b.shape=(100, 100), m=8, n=100, k=100
	 [[{{node sequential_1/lstm_1/while/body/_1/sequential_1/lstm_1/while/lstm_cell_1/MatMul_6}}]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_8031]

Function call stack:
train_function -> train_function


In [13]:
y_pred= model.predict(X_te)
y_pred = np.argmax(y_pred, axis=1)
y_test = test_data['#classes_id']
Y_te = pd.get_dummies(y_test).values

Y_te = np.argmax(Y_te, axis=1)
print(f1_score(Y_te, y_pred, average='macro'))


0.14653626397278355


In [15]:
target_names = list(train_data["#3_country_label"].unique())
print(classification_report(Y_te, y_pred,target_names=target_names))

                      precision    recall  f1-score   support

               Egypt       0.52      0.60      0.56      1041
                Iraq       0.38      0.41      0.39       663
        Saudi_Arabia       0.25      0.27      0.26       519
          Mauritania       0.20      0.17      0.18        53
             Algeria       0.38      0.34      0.36       430
               Syria       0.10      0.11      0.11       278
                Oman       0.17      0.15      0.16       355
             Tunisia       0.14      0.15      0.15       172
             Lebanon       0.10      0.09      0.09       157
             Morocco       0.08      0.09      0.08       207
            Djibouti       0.00      0.00      0.00        27
United_Arab_Emirates       0.08      0.08      0.08       157
              Kuwait       0.06      0.06      0.06       105
               Libya       0.28      0.23      0.25       314
             Bahrain       0.00      0.00      0.00        52
       